# FIN Model

In [5]:
import numpy as np
from numpy import random
import pandas as pd
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D

In [6]:
random.seed(222)

# Task 1

Train two feature imitating networks that take as input a fixed length sequence and provide as output (1) the mean and (2) the standard deviation of a fixed length input; a deep feedforward architecture should be sufficient for this.

In [7]:
# Z-scored np array
# T x Ch x Epochs
N = 100 # Fixed number of length data
E = 3000 # Size
input_arr = np.random.randint(10, size=(N,1, E) )
scaled_arr = np.zeros(input_arr.shape)

#print(input_arr)

for ii in range(input_arr.shape[1]):
#     scaler = StandardScaler()
#     scaled_arr[:,ii,:] = scaler.fit_transform(input_arr[:,ii,:])
    scaled_arr[:,ii,:] = input_arr[:,ii,:]
    
#print(scaled_arr.shape)
#print(scaled_arr)

# move the axis of the matrix as as # Epochs X T X Ch

scaled_arr = np.moveaxis(scaled_arr,2,0)
print(scaled_arr.shape)

do_cwt = False
if do_cwt:
    # Continuous wavelet transform
    flat_data = np.squeeze(scaled_arr)
    scaled_arr_cwt = []
    for ii in range(flat_data.shape[0]):
        widths = np.arange(1, 31)
        cwtmatr = cwt(flat_data[ii], ricker, widths)
        scaled_arr_cwt.append(np.moveaxis(cwtmatr,1,0))
    scaled_arr3d = np.stack(scaled_arr_cwt)
else:
    scaled_arr3d = scaled_arr
    
print(scaled_arr)



(3000, 100, 1)
[[[6.]
  [6.]
  [7.]
  ...
  [8.]
  [0.]
  [2.]]

 [[3.]
  [9.]
  [4.]
  ...
  [6.]
  [1.]
  [9.]]

 [[2.]
  [6.]
  [2.]
  ...
  [3.]
  [4.]
  [6.]]

 ...

 [[2.]
  [5.]
  [9.]
  ...
  [6.]
  [5.]
  [4.]]

 [[8.]
  [1.]
  [9.]
  ...
  [3.]
  [7.]
  [0.]]

 [[4.]
  [2.]
  [0.]
  ...
  [5.]
  [2.]
  [5.]]]


In [8]:
# Will use Mean for training, so calculate the Mean of the data
def Mean(data):
    return np.mean(data,axis=1)

def STD(data):
    return np.std(data,axis=1)

#squeeze the data in
feature_res = np.squeeze(Mean(scaled_arr))
feature_res_std = np.squeeze(STD(scaled_arr))

# the predict result should be
# print(np.mean(scaled_arr))
# print(feature_res)
# print(feature_res.shape)
print(feature_res_std.shape)

(3000,)


In [9]:
# Make FIN
def make_FIN(topology,size3D,N,name="my-FIN"):
    input_layer = keras.Input(shape=(N,size3D))
    x = Flatten()(input_layer)
    for layer in topology['layers']:
        x = Dense(layer['output_shape'], activation='relu')(x)
        if layer['BatchNorm']:
            x = BatchNormalization(axis=1)(x)
    fin_out = Dense(1, activation='linear')(x)
    fin = keras.Model(input_layer, fin_out, name=name)
    return fin

topology = np.load('RandomTopology.npy', allow_pickle=True)[1]


fin = make_FIN(topology,1,N)
fin2 = make_FIN(topology,1,N)


In [10]:
#learning_rate=topology['lr']
learning_rate= 0.0005
momentum=topology['momentum']
nesterov=topology['nesterov']
b1 = topology['b1']
b2 = topology['b2']
if topology['opt'] == 'SGD':
    grad_desc_algorithm = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov)
if topology['opt'] == 'ADAM':
    grad_desc_algorithm = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=b1,beta_2=b2)
    
#es = EarlyStopping(monitor='accuracy', mode='min', verbose=1,patience=3)

fin.compile(optimizer = grad_desc_algorithm,loss = 'mean_squared_error')
fin2.compile(optimizer = grad_desc_algorithm,loss = 'mean_squared_error')


loss_d = fin.fit(scaled_arr3d, feature_res, 
            epochs = 1000,
            batch_size = 500,
            verbose   = 1)    

loss_d2 = fin2.fit(scaled_arr3d, feature_res_std, 
            epochs = 1000,
            batch_size = 500,
            verbose   = 1)   


Epoch 1/1000
6/6 [==============================] - 2s 68ms/step - loss: 19.3242
Epoch 2/1000
6/6 [==============================] - 1s 125ms/step - loss: 13.4569
Epoch 3/1000
6/6 [==============================] - 1s 143ms/step - loss: 4.6924
Epoch 4/1000
6/6 [==============================] - 0s 67ms/step - loss: 2.1983
Epoch 5/1000
6/6 [==============================] - 1s 96ms/step - loss: 1.2749
Epoch 6/1000
6/6 [==============================] - 1s 77ms/step - loss: 0.8936
Epoch 7/1000
6/6 [==============================] - 0s 52ms/step - loss: 0.7243
Epoch 8/1000
6/6 [==============================] - 0s 95ms/step - loss: 0.6005
Epoch 9/1000
6/6 [==============================] - 1s 97ms/step - loss: 0.5889
Epoch 10/1000
6/6 [==============================] - 1s 109ms/step - loss: 0.4988
Epoch 11/1000
6/6 [==============================] - 1s 95ms/step - loss: 0.4591
Epoch 12/1000
6/6 [==============================] - 0s 70ms/step - loss: 0.4087
Epoch 13/1000
6/6 [=============

6/6 [==============================] - 0s 72ms/step - loss: 0.0017
Epoch 102/1000
6/6 [==============================] - 0s 58ms/step - loss: 0.0019
Epoch 103/1000
6/6 [==============================] - 0s 44ms/step - loss: 0.0038
Epoch 104/1000
6/6 [==============================] - 0s 59ms/step - loss: 0.0096
Epoch 105/1000
6/6 [==============================] - 0s 72ms/step - loss: 0.0087
Epoch 106/1000
6/6 [==============================] - 0s 71ms/step - loss: 0.0041
Epoch 107/1000
6/6 [==============================] - 1s 169ms/step - loss: 0.0127
Epoch 108/1000
6/6 [==============================] - 1s 74ms/step - loss: 0.0058
Epoch 109/1000
6/6 [==============================] - 1s 97ms/step - loss: 0.0049
Epoch 110/1000
6/6 [==============================] - 1s 98ms/step - loss: 0.0033
Epoch 111/1000
6/6 [==============================] - 1s 86ms/step - loss: 0.0028
Epoch 112/1000
6/6 [==============================] - 1s 90ms/step - loss: 0.0112
Epoch 113/1000
6/6 [==========

6/6 [==============================] - 1s 116ms/step - loss: 0.0056
Epoch 201/1000
6/6 [==============================] - 1s 103ms/step - loss: 0.0065
Epoch 202/1000
6/6 [==============================] - 1s 125ms/step - loss: 0.0029
Epoch 203/1000
6/6 [==============================] - 1s 100ms/step - loss: 0.0028
Epoch 204/1000
6/6 [==============================] - 1s 100ms/step - loss: 0.0028
Epoch 205/1000
6/6 [==============================] - 1s 107ms/step - loss: 0.0033
Epoch 206/1000
6/6 [==============================] - 1s 91ms/step - loss: 0.0011
Epoch 207/1000
6/6 [==============================] - 1s 123ms/step - loss: 0.0023
Epoch 208/1000
6/6 [==============================] - 1s 96ms/step - loss: 0.0059
Epoch 209/1000
6/6 [==============================] - 1s 88ms/step - loss: 0.0054
Epoch 210/1000
6/6 [==============================] - 1s 121ms/step - loss: 0.0024
Epoch 211/1000
6/6 [==============================] - 1s 127ms/step - loss: 0.0012
Epoch 212/1000
6/6 [==

6/6 [==============================] - 0s 47ms/step - loss: 0.0016
Epoch 300/1000
6/6 [==============================] - 0s 60ms/step - loss: 0.0011
Epoch 301/1000
6/6 [==============================] - 0s 44ms/step - loss: 0.0032
Epoch 302/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.0060
Epoch 303/1000
6/6 [==============================] - 0s 49ms/step - loss: 0.0020
Epoch 304/1000
6/6 [==============================] - 0s 48ms/step - loss: 0.0020
Epoch 305/1000
6/6 [==============================] - 0s 71ms/step - loss: 0.0026
Epoch 306/1000
6/6 [==============================] - 0s 44ms/step - loss: 0.0014
Epoch 307/1000
6/6 [==============================] - 0s 81ms/step - loss: 0.0021
Epoch 308/1000
6/6 [==============================] - 0s 66ms/step - loss: 0.0012
Epoch 309/1000
6/6 [==============================] - 1s 91ms/step - loss: 0.0034
Epoch 310/1000
6/6 [==============================] - 0s 82ms/step - loss: 0.0025
Epoch 311/1000
6/6 [===========

6/6 [==============================] - 0s 82ms/step - loss: 0.0048
Epoch 399/1000
6/6 [==============================] - 1s 94ms/step - loss: 0.0084
Epoch 400/1000
6/6 [==============================] - 0s 35ms/step - loss: 0.0046
Epoch 401/1000
6/6 [==============================] - 0s 73ms/step - loss: 0.0012
Epoch 402/1000
6/6 [==============================] - 1s 103ms/step - loss: 9.4502e-04
Epoch 403/1000
6/6 [==============================] - 0s 71ms/step - loss: 0.0021
Epoch 404/1000
6/6 [==============================] - 1s 99ms/step - loss: 0.0033
Epoch 405/1000
6/6 [==============================] - 1s 90ms/step - loss: 0.0031
Epoch 406/1000
6/6 [==============================] - 0s 31ms/step - loss: 0.0044
Epoch 407/1000
6/6 [==============================] - 0s 63ms/step - loss: 0.0053
Epoch 408/1000
6/6 [==============================] - 0s 80ms/step - loss: 0.0010
Epoch 409/1000
6/6 [==============================] - 1s 98ms/step - loss: 0.0020
Epoch 410/1000
6/6 [======

...

6/6 [==============================] - 0s 17ms/step - loss: 4.4276e-04
Epoch 580/1000
6/6 [==============================] - 0s 18ms/step - loss: 5.3048e-04
Epoch 581/1000
6/6 [==============================] - 0s 70ms/step - loss: 5.5477e-04
Epoch 582/1000
6/6 [==============================] - 0s 48ms/step - loss: 9.0220e-04
Epoch 583/1000
6/6 [==============================] - 0s 47ms/step - loss: 7.2728e-04
Epoch 584/1000
6/6 [==============================] - 0s 35ms/step - loss: 0.0025
Epoch 585/1000
6/6 [==============================] - 0s 41ms/step - loss: 0.0017
Epoch 586/1000
6/6 [==============================] - 0s 39ms/step - loss: 9.6560e-04
Epoch 587/1000
6/6 [==============================] - 0s 66ms/step - loss: 8.2935e-04
Epoch 588/1000
6/6 [==============================] - 0s 49ms/step - loss: 6.1161e-04
Epoch 589/1000
6/6 [==============================] - 0s 37ms/step - loss: 5.2263e-04
Epoch 590/1000
6/6 [==============================] - 0s 44ms/step - loss: 6.

6/6 [==============================] - 0s 34ms/step - loss: 0.0013
Epoch 676/1000
6/6 [==============================] - 0s 37ms/step - loss: 6.6145e-04
Epoch 677/1000
6/6 [==============================] - 0s 42ms/step - loss: 5.7091e-04
Epoch 678/1000
6/6 [==============================] - 0s 44ms/step - loss: 0.0014
Epoch 679/1000
6/6 [==============================] - 0s 22ms/step - loss: 5.4955e-04
Epoch 680/1000
6/6 [==============================] - 0s 31ms/step - loss: 0.0011
Epoch 681/1000
6/6 [==============================] - 0s 37ms/step - loss: 6.2565e-04
Epoch 682/1000
6/6 [==============================] - 0s 50ms/step - loss: 5.1306e-04
Epoch 683/1000
6/6 [==============================] - 0s 33ms/step - loss: 4.1914e-04
Epoch 684/1000
6/6 [==============================] - 0s 28ms/step - loss: 0.0013
Epoch 685/1000
6/6 [==============================] - 0s 37ms/step - loss: 0.0012
Epoch 686/1000
6/6 [==============================] - 0s 60ms/step - loss: 0.0013
Epoch 6

6/6 [==============================] - 0s 39ms/step - loss: 8.5871e-04
Epoch 772/1000
6/6 [==============================] - 0s 43ms/step - loss: 3.8933e-04
Epoch 773/1000
6/6 [==============================] - 0s 53ms/step - loss: 9.4502e-04
Epoch 774/1000
6/6 [==============================] - 0s 35ms/step - loss: 4.6403e-04
Epoch 775/1000
6/6 [==============================] - 0s 37ms/step - loss: 5.9345e-04
Epoch 776/1000
6/6 [==============================] - 0s 58ms/step - loss: 8.4716e-04
Epoch 777/1000
6/6 [==============================] - 0s 57ms/step - loss: 4.2638e-04
Epoch 778/1000
6/6 [==============================] - 0s 37ms/step - loss: 6.8211e-04
Epoch 779/1000
6/6 [==============================] - 0s 45ms/step - loss: 9.7981e-04
Epoch 780/1000
6/6 [==============================] - 0s 47ms/step - loss: 0.0013
Epoch 781/1000
6/6 [==============================] - 0s 32ms/step - loss: 4.6620e-04
Epoch 782/1000
6/6 [==============================] - 0s 59ms/step - loss

6/6 [==============================] - 0s 84ms/step - loss: 0.0018
Epoch 868/1000
6/6 [==============================] - 0s 73ms/step - loss: 0.0010
Epoch 869/1000
6/6 [==============================] - 0s 73ms/step - loss: 9.9981e-04
Epoch 870/1000
6/6 [==============================] - 0s 84ms/step - loss: 0.0015
Epoch 871/1000
6/6 [==============================] - 0s 62ms/step - loss: 6.7051e-04
Epoch 872/1000
6/6 [==============================] - 0s 84ms/step - loss: 3.5486e-04
Epoch 873/1000
6/6 [==============================] - 0s 71ms/step - loss: 5.7404e-04
Epoch 874/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.0015
Epoch 875/1000
6/6 [==============================] - 0s 48ms/step - loss: 6.7563e-04
Epoch 876/1000
6/6 [==============================] - 0s 63ms/step - loss: 5.1159e-04
Epoch 877/1000
6/6 [==============================] - 0s 35ms/step - loss: 3.5764e-04
Epoch 878/1000
6/6 [==============================] - 0s 37ms/step - loss: 7.7816e-04

6/6 [==============================] - 1s 83ms/step - loss: 5.4214e-04
Epoch 964/1000
6/6 [==============================] - 1s 92ms/step - loss: 5.9815e-04
Epoch 965/1000
6/6 [==============================] - 0s 79ms/step - loss: 6.5445e-04
Epoch 966/1000
6/6 [==============================] - 1s 88ms/step - loss: 5.8261e-04
Epoch 967/1000
6/6 [==============================] - 0s 56ms/step - loss: 7.8165e-04
Epoch 968/1000
6/6 [==============================] - 0s 43ms/step - loss: 8.0953e-04
Epoch 969/1000
6/6 [==============================] - 0s 76ms/step - loss: 4.4428e-04
Epoch 970/1000
6/6 [==============================] - 0s 83ms/step - loss: 0.0018
Epoch 971/1000
6/6 [==============================] - 1s 111ms/step - loss: 0.0011
Epoch 972/1000
6/6 [==============================] - 1s 98ms/step - loss: 4.9994e-04
Epoch 973/1000
6/6 [==============================] - 1s 88ms/step - loss: 7.4638e-04
Epoch 974/1000
6/6 [==============================] - 1s 87ms/step - loss: 4

In [26]:
Predirect_arr = np.random.randint(10, size=(1,100, 1)).T

# fin.predict(Predirect_arr)
print(np.mean(Predirect_arr))
print(fin.predict(Predirect_arr))
print(np.std(Predirect_arr))
print(fin2.predict(Predirect_arr))


4.41
[[4.4220705]]
2.7279846040621267
[[2.7919717]]


In [19]:
#fin.save("./Models/Modelstask1Mean.h5")
fin2.save("./Models/Modelstask1Std.h5")

In [20]:
task1MeanModel = load_model('./Models/Modelstask1Mean.h5')


In [21]:
Predirect_arr = np.random.randint(10, size=(1,100, 1)).T
print(np.mean(Predirect_arr))
task1MeanModel.predict(Predirect_arr)


4.91


array([[5.017105]], dtype=float32)